In [ ]:
# Célula 1: Imports e Configuração
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from tqdm import tqdm
import warnings
import json
from collections import defaultdict
import traceback
import matplotlib.pyplot as plt
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

# Configuração
BASE_PATH = r'../data/raw/RAF-DB/DATASET'
OUTPUT_PATH = r'../data/processed/RAF-DB'
TARGET_SIZE = (224, 224)
LOG_PATH = r'../data/logs'

print("🔧 Configuração do Pré-processamento")
print("="*60)
print(f"📁 Input:  {BASE_PATH}")
print(f"📁 Output: {OUTPUT_PATH}")
print(f"📁 Logs:   {LOG_PATH}")
print(f"📐 Target Size: {TARGET_SIZE}")
print("="*60)

In [ ]:
# Célula 2: Verificação de Ambiente
import sys
import subprocess

# Verificar instalações necessárias
required_packages = ['cv2', 'mtcnn', 'tensorflow', 'tqdm', 'matplotlib']

print("📦 Verificando pacotes necessários...")
for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} instalado")
    except ImportError:
        print(f"❌ {package} não encontrado")
        print(f"   Execute: pip install {package}")

In [ ]:
# Célula 3: Verificação de Caminhos
# Verificar se o caminho base existe
if not os.path.exists(BASE_PATH):
    print(f"❌ ERRO: Caminho base não encontrado: {BASE_PATH}")
    print(f"\n📂 Estrutura de diretórios esperada:")
    print("""
    data/
    └── raw/
        └── RAF-DB/
            ├── train/
            │   ├── anger/
            │   ├── disgust/
            │   ├── fear/
            │   ├── happiness/
            │   ├── neutral/
            │   ├── sadness/
            │   └── surprise/
            └── test/
                ├── anger/
                ├── disgust/
                ├── fear/
                ├── happiness/
                ├── neutral/
                ├── sadness/
                └── surprise/
    """)
    
    print(f"\n🔍 Procurando por RAF-DB no diretório atual...")
    found = False
    for root, dirs, files in os.walk('.'):
        if 'RAF-DB' in root or 'raf-db' in root.lower():
            print(f"  📁 Encontrado: {root}")
            found = True
    
    if not found:
        print("  ❌ Nenhum diretório RAF-DB encontrado")
else:
    print(f"✅ Caminho base encontrado: {BASE_PATH}")
    
    # Criar pastas necessárias
    os.makedirs(LOG_PATH, exist_ok=True)
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    print(f"✅ Pastas de saída criadas")

In [ ]:
# Célula 4: Inicialização do MTCNN
# Inicializar o detector
detector = None
try:
    # Tentar inicialização com parâmetros completos
    try:
        detector = MTCNN(
            min_face_size=20,
            scale_factor=0.709,
            steps_threshold=[0.6, 0.7, 0.7]
        )
        print("✅ MTCNN inicializado com sucesso (com parâmetros)")
    except TypeError:
        # Fallback: inicialização padrão sem parâmetros
        detector = MTCNN()
        print("✅ MTCNN inicializado com sucesso (parâmetros padrão)")
    
    print("\nDetector MTCNN pronto para uso!")
    print("  • Biblioteca: mtcnn")
    print("  • Backend: TensorFlow/Keras")
    
except Exception as e:
    print(f"❌ ERRO ao inicializar MTCNN: {e}")
    print("\n🔧 Solução:")
    print("  1. Desinstale versões conflitantes:")
    print("     pip uninstall mtcnn")
    print("  2. Instale a versão correta:")
    print("     pip install mtcnn")
    print("  3. Verifique se o TensorFlow está instalado:")
    print("     pip install tensorflow")

In [ ]:
# Célula 5: Estrutura de Logging
# Estrutura para logging
processing_stats = defaultdict(lambda: {
    'total': 0,
    'processed': 0,
    'failed_read': [],
    'no_face_detected': [],
    'alignment_failed': [],
    'crop_failed': [],
    'other_errors': []
})

print("📊 Sistema de logging inicializado")

In [ ]:
# Célula 6: Funções Auxiliares
def validate_image(img_path):
    """
    Valida se o arquivo é uma imagem válida.
    """
    try:
        img = cv2.imread(img_path)
        if img is None:
            return False, None
        if img.shape[0] < 10 or img.shape[1] < 10:
            return False, None
        return True, img
    except Exception:
        return False, None

def safe_crop(image, bbox, padding=0.2):
    """
    Recorta face com padding e tratamento de bordas.
    """
    try:
        x, y, w, h = bbox
        
        # Validar bbox
        if w <= 0 or h <= 0:
            return None
            
        height, width = image.shape[:2]
        
        # Adicionar padding
        pad_w = int(w * padding)
        pad_h = int(h * padding)
        
        # Calcular nova bbox com padding
        x1 = max(0, x - pad_w)
        y1 = max(0, y - pad_h)
        x2 = min(width, x + w + pad_w)
        y2 = min(height, y + h + pad_h)
        
        # Validar coordenadas
        if x2 <= x1 or y2 <= y1:
            return None
            
        cropped = image[y1:y2, x1:x2]
        
        if cropped.size == 0 or cropped.shape[0] < 10 or cropped.shape[1] < 10:
            return None
        
        return cropped
    except Exception as e:
        print(f"Erro no crop: {e}")
        return None

def align_face_safe(image, left_eye, right_eye, expand_factor=1.2):
    """
    Alinhamento com expansão de canvas para evitar cortes.
    """
    try:
        # Validar coordenadas dos olhos
        if left_eye[0] == right_eye[0] and left_eye[1] == right_eye[1]:
            return image  # Olhos no mesmo ponto, retornar original
            
        eye_dx = right_eye[0] - left_eye[0]
        eye_dy = right_eye[1] - left_eye[1]
        angle = np.degrees(np.arctan2(eye_dy, eye_dx))
        
        # Se o ângulo é muito pequeno, não rotacionar
        if abs(angle) < 1.0:
            return image
        
        (h, w) = image.shape[:2]
        
        # Limitar o fator de expansão
        expand_factor = min(expand_factor, 1.5)
        
        # Expandir canvas para evitar cortes durante rotação
        new_h = int(h * expand_factor)
        new_w = int(w * expand_factor)
        
        # Criar imagem expandida
        if len(image.shape) == 3:
            expanded = np.zeros((new_h, new_w, 3), dtype=np.uint8)
        else:
            expanded = np.zeros((new_h, new_w), dtype=np.uint8)
            
        y_offset = (new_h - h) // 2
        x_offset = (new_w - w) // 2
        expanded[y_offset:y_offset+h, x_offset:x_offset+w] = image
        
        # Centro da imagem expandida
        center = (new_w // 2, new_h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        aligned = cv2.warpAffine(expanded, M, (new_w, new_h), 
                                 flags=cv2.INTER_CUBIC,
                                 borderMode=cv2.BORDER_REPLICATE)
        
        return aligned
    except Exception as e:
        print(f"Erro no alinhamento: {e}")
        return image

print("✅ Funções auxiliares carregadas")

In [ ]:
# Célula 7: Função Principal de Processamento
def process_with_fallback(img_path, output_path, emotion, split):
    """
    Processa imagem com múltiplas estratégias de fallback.
    """
    stats_key = f"{split}/{emotion}"
    stats = processing_stats[stats_key]
    stats['total'] += 1
    
    # Validar e ler imagem
    is_valid, img = validate_image(img_path)
    if not is_valid:
        stats['failed_read'].append(os.path.basename(img_path))
        return False
    
    try:
        # Estratégia 1: Detecção padrão
        results = detector.detect_faces(img)
        
        # Estratégia 2: Se falhar, tentar com imagem equalizada
        if not results and len(img.shape) == 3:
            gray_temp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            equalized = cv2.equalizeHist(gray_temp)
            img_eq = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
            results = detector.detect_faces(img_eq)
            if results:
                img = img_eq  # Usar imagem equalizada
            
        # Estratégia 3: Se ainda falhar, tentar com CLAHE
        if not results and len(img.shape) == 3:
            lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
            clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
            l_clahe = clahe.apply(l)
            lab_clahe = cv2.merge([l_clahe, a, b])
            img_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
            results = detector.detect_faces(img_clahe)
            if results:
                img = img_clahe
        
        if not results:
            stats['no_face_detected'].append(os.path.basename(img_path))
            
            # Fallback: salvar crop central
            h, w = img.shape[:2]
            center_size = min(h, w) // 2
            y_start = (h - center_size) // 2
            x_start = (w - center_size) // 2
            center_crop = img[y_start:y_start+center_size, x_start:x_start+center_size]
            
            if center_crop.size > 0:
                resized = cv2.resize(center_crop, TARGET_SIZE, interpolation=cv2.INTER_AREA)
                if len(resized.shape) == 3:
                    gray_face = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
                else:
                    gray_face = resized
                cv2.imwrite(output_path, gray_face)
                stats['processed'] += 1
                return True
            return False
        
        # Processar com detecção bem-sucedida
        face_data = results[0]
        
        # Verificar se tem keypoints para alinhamento
        if 'keypoints' in face_data:
            keypoints = face_data['keypoints']
            if 'left_eye' in keypoints and 'right_eye' in keypoints:
                left_eye = keypoints['left_eye']
                right_eye = keypoints['right_eye']
                
                # Alinhamento seguro
                aligned_img = align_face_safe(img, left_eye, right_eye)
                
                # Re-detectar na imagem alinhada
                results_aligned = detector.detect_faces(aligned_img)
                
                if results_aligned:
                    face_img = safe_crop(aligned_img, results_aligned[0]['box'])
                else:
                    face_img = safe_crop(img, face_data['box'])
            else:
                face_img = safe_crop(img, face_data['box'])
        else:
            face_img = safe_crop(img, face_data['box'])
        
        if face_img is None:
            stats['crop_failed'].append(os.path.basename(img_path))
            return False
        
        # Redimensionar e converter para escala de cinza
        resized = cv2.resize(face_img, TARGET_SIZE, interpolation=cv2.INTER_AREA)
        
        if len(resized.shape) == 3:
            gray_face = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
        else:
            gray_face = resized
        
        # Salvar imagem processada
        success = cv2.imwrite(output_path, gray_face)
        if success:
            stats['processed'] += 1
            return True
        else:
            stats['other_errors'].append({
                'file': os.path.basename(img_path),
                'error': 'Falha ao salvar imagem'
            })
            return False
        
    except Exception as e:
        stats['other_errors'].append({
            'file': os.path.basename(img_path),
            'error': str(e),
            'traceback': traceback.format_exc()
        })
        return False

print("✅ Função de processamento carregada")

In [ ]:
# Célula 8: Análise da Estrutura do Dataset
print("="*60)
print("📊 ANÁLISE DA ESTRUTURA DO DATASET")
print("="*60)

dataset_info = {}

# Verificar estrutura esperada
for split in ['train', 'test']:
    split_path = os.path.join(BASE_PATH, split)
    if os.path.exists(split_path):
        emotions = [d for d in os.listdir(split_path) 
                   if os.path.isdir(os.path.join(split_path, d))]
        
        dataset_info[split] = {}
        print(f"\n✅ {split.upper()}: {len(emotions)} emoções encontradas")
        
        total_split = 0
        for emotion in emotions:
            emotion_path = os.path.join(split_path, emotion)
            n_images = len([f for f in os.listdir(emotion_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
            dataset_info[split][emotion] = n_images
            total_split += n_images
            print(f"   • {emotion:12s}: {n_images:6d} imagens")
        
        print(f"   {'─'*30}")
        print(f"   {'TOTAL':12s}: {total_split:6d} imagens")
    else:
        print(f"\n❌ {split.upper()}: pasta não encontrada")

In [ ]:
# Célula 9: Visualização da Distribuição (Opcional)
if dataset_info:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, len(dataset_info), figsize=(14, 6))
    if len(dataset_info) == 1:
        axes = [axes]
    
    colors = plt.cm.Set3(np.linspace(0, 1, 7))
    
    for idx, (split, emotions_data) in enumerate(dataset_info.items()):
        ax = axes[idx]
        emotions = list(emotions_data.keys())
        counts = list(emotions_data.values())
        
        bars = ax.bar(emotions, counts, color=colors)
        ax.set_title(f'{split.upper()} Set Distribution')
        ax.set_xlabel('Emotion')
        ax.set_ylabel('Number of Images')
        ax.tick_params(axis='x', rotation=45)
        
        # Adicionar valores nas barras
        for bar, count in zip(bars, counts):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{count}', ha='center', va='bottom')
    
    plt.suptitle('RAF-DB Dataset Distribution', fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()

In [ ]:
# Célula 10: Processamento Principal
print("\n" + "="*60)
print("🚀 INICIANDO PROCESSAMENTO...")
print("="*60)

if not os.path.exists(BASE_PATH):
    print("⚠️ Não é possível processar - caminho base não existe")
else:
    # Processar imagens
    for split in ['train', 'test']:
        input_split_path = os.path.join(BASE_PATH, split)
        output_split_path = os.path.join(OUTPUT_PATH, split)
        
        if not os.path.exists(input_split_path):
            print(f"\n⚠️  Pulando {split}: pasta não encontrada")
            continue
        
        emotions = [d for d in os.listdir(input_split_path) 
                    if os.path.isdir(os.path.join(input_split_path, d))]
        
        print(f"\n📂 Processando {split}...")
        
        # Usar tqdm notebook para melhor visualização
        from tqdm.notebook import tqdm as tqdm_notebook
        
        for emotion in tqdm_notebook(emotions, desc=f"{split}"):
            input_emotion_path = os.path.join(input_split_path, emotion)
            output_emotion_path = os.path.join(output_split_path, emotion)
            os.makedirs(output_emotion_path, exist_ok=True)
            
            images = [f for f in os.listdir(input_emotion_path) 
                     if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            
            # Processar com sub-progress bar
            for image_name in tqdm_notebook(images, desc=f"  {emotion}", leave=False):
                input_path = os.path.join(input_emotion_path, image_name)
                output_path = os.path.join(output_emotion_path, image_name)
                
                # Pular se já foi processada
                if os.path.exists(output_path):
                    continue
                    
                process_with_fallback(input_path, output_path, emotion, split)
    
    print("\n✅ Processamento concluído!")

In [ ]:
# Célula 11: Salvar Estatísticas
# Converter defaultdict para dict regular para serialização
stats_to_save = {}
for key, value in processing_stats.items():
    stats_to_save[key] = dict(value)

# Salvar estatísticas em JSON
stats_file = os.path.join(LOG_PATH, 'raf_db_preprocessing_stats.json')
with open(stats_file, 'w') as f:
    json.dump(stats_to_save, f, indent=2, default=str)

print(f"💾 Estatísticas salvas em: {stats_file}")

# Também salvar um CSV para análise
import pandas as pd

stats_summary = []
for key, stats in processing_stats.items():
    split, emotion = key.split('/')
    stats_summary.append({
        'split': split,
        'emotion': emotion,
        'total': stats['total'],
        'processed': stats['processed'],
        'failed_read': len(stats['failed_read']),
        'no_face': len(stats['no_face_detected']),
        'crop_failed': len(stats['crop_failed']),
        'other_errors': len(stats['other_errors'])
    })

if stats_summary:
    df_stats = pd.DataFrame(stats_summary)
    csv_file = os.path.join(LOG_PATH, 'raf_db_preprocessing_summary.csv')
    df_stats.to_csv(csv_file, index=False)
    print(f"📊 Sumário salvo em: {csv_file}")

In [ ]:
# Célula 12: Relatório Final Detalhado
print("\n" + "="*60)
print("📊 RELATÓRIO DE PROCESSAMENTO DETALHADO")
print("="*60)

total_images = 0
total_processed = 0
total_failures = defaultdict(int)

# Criar DataFrame para visualização
results_data = []

for key, stats in processing_stats.items():
    if stats['total'] == 0:
        continue
        
    split, emotion = key.split('/')
    total = stats['total']
    processed = stats['processed']
    
    success_rate = (processed/total*100) if total > 0 else 0
    status = "✅" if success_rate > 90 else "⚠️" if success_rate > 70 else "❌"
    
    results_data.append({
        'Split': split,
        'Emotion': emotion,
        'Status': status,
        'Total': total,
        'Processed': processed,
        'Success Rate': f"{success_rate:.1f}%",
        'Failed Read': len(stats['failed_read']),
        'No Face': len(stats['no_face_detected']),
        'Crop Failed': len(stats['crop_failed']),
        'Other Errors': len(stats['other_errors'])
    })
    
    # Acumular totais
    total_images += total
    total_processed += processed
    
    if stats['failed_read']:
        total_failures['failed_read'] += len(stats['failed_read'])
    if stats['no_face_detected']:
        total_failures['no_face'] += len(stats['no_face_detected'])
    if stats['crop_failed']:
        total_failures['crop'] += len(stats['crop_failed'])
    if stats['other_errors']:
        total_failures['other'] += len(stats['other_errors'])

# Mostrar tabela de resultados
if results_data:
    df_results = pd.DataFrame(results_data)
    display(df_results.style.set_properties(**{'text-align': 'center'}))

In [ ]:
# Célula 13: Resumo Geral e Gráficos
if total_images > 0:
    print("\n" + "="*60)
    print("📈 RESUMO GERAL")
    print("="*60)
    print(f"Total de imagens: {total_images}")
    print(f"✅ Processadas: {total_processed} ({total_processed/total_images*100:.1f}%)")
    print(f"❌ Perdidas: {total_images - total_processed} ({(total_images-total_processed)/total_images*100:.1f}%)")
    
    # Gráfico de pizza para causas de falha
    if total_failures:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # Gráfico 1: Sucesso vs Falha
        success_data = [total_processed, total_images - total_processed]
        success_labels = ['Processadas', 'Perdidas']
        colors1 = ['#2ecc71', '#e74c3c']
        
        ax1.pie(success_data, labels=success_labels, colors=colors1, 
                autopct='%1.1f%%', startangle=90)
        ax1.set_title('Taxa de Sucesso do Processamento')
        
        # Gráfico 2: Distribuição de Falhas
        if sum(total_failures.values()) > 0:
            failure_values = list(total_failures.values())
            failure_labels = list(total_failures.keys())
            colors2 = plt.cm.Set3(np.linspace(0, 1, len(failure_labels)))
            
            ax2.pie(failure_values, labels=failure_labels, colors=colors2, 
                    autopct='%1.1f%%', startangle=90)
            ax2.set_title('Distribuição das Causas de Falha')
        
        plt.suptitle('Análise do Processamento RAF-DB', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        print("\n📉 Principais causas de perda:")
        for cause, count in sorted(total_failures.items(), key=lambda x: x[1], reverse=True):
            print(f"   • {cause}: {count} ({count/total_images*100:.1f}%)")

In [ ]:
# Célula 14: Visualização de Exemplos Processados (Opcional)
def show_processing_examples(n_examples=5):
    """
    Mostra exemplos de imagens antes e depois do processamento
    """
    print("🖼️ Exemplos de Processamento")
    print("="*60)
    
    processed_examples = []
    
    # Buscar exemplos processados
    for split in ['train', 'test']:
        output_split_path = os.path.join(OUTPUT_PATH, split)
        if not os.path.exists(output_split_path):
            continue
            
        for emotion in os.listdir(output_split_path):
            emotion_path = os.path.join(output_split_path, emotion)
            if not os.path.isdir(emotion_path):
                continue
                
            for img_file in os.listdir(emotion_path)[:2]:  # Pegar 2 de cada emoção
                if img_file.lower().endswith(('.jpg', '.png')):
                    processed_examples.append({
                        'path': os.path.join(emotion_path, img_file),
                        'emotion': emotion,
                        'split': split
                    })
                    
                if len(processed_examples) >= n_examples:
                    break
            
            if len(processed_examples) >= n_examples:
                break
    
    # Mostrar exemplos
    if processed_examples:
        fig, axes = plt.subplots(1, min(n_examples, len(processed_examples)), 
                                 figsize=(15, 4))
        if len(processed_examples) == 1:
            axes = [axes]
        
        for idx, example in enumerate(processed_examples[:n_examples]):
            img = cv2.imread(example['path'], cv2.IMREAD_GRAYSCALE)
            if img is not None:
                axes[idx].imshow(img, cmap='gray')
                axes[idx].set_title(f"{example['emotion']}\n({example['split']})")
                axes[idx].axis('off')
        
        plt.suptitle('Exemplos de Faces Processadas (224x224, Grayscale)', fontsize=14)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Nenhum exemplo processado encontrado")

# Chamar a função
if os.path.exists(OUTPUT_PATH):
    show_processing_examples()

In [ ]:
# Célula 15: Análise de Qualidade (Opcional)
def analyze_processing_quality():
    """
    Analisa a qualidade das imagens processadas
    """
    print("\n🔍 Análise de Qualidade")
    print("="*60)
    
    quality_metrics = []
    
    for split in ['train', 'test']:
        output_split_path = os.path.join(OUTPUT_PATH, split)
        if not os.path.exists(output_split_path):
            continue
        
        for emotion in os.listdir(output_split_path):
            emotion_path = os.path.join(output_split_path, emotion)
            if not os.path.isdir(emotion_path):
                continue
            
            # Amostrar algumas imagens para análise
            sample_images = [f for f in os.listdir(emotion_path) 
                           if f.lower().endswith(('.jpg', '.png'))][:10]
            
            for img_file in sample_images:
                img_path = os.path.join(emotion_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                if img is not None:
                    # Calcular métricas
                    quality_metrics.append({
                        'split': split,
                        'emotion': emotion,
                        'brightness': np.mean(img),
                        'contrast': np.std(img),
                        'sharpness': cv2.Laplacian(img, cv2.CV_64F).var()
                    })
    
    if quality_metrics:
        df_quality = pd.DataFrame(quality_metrics)
        
        # Estatísticas gerais
        print("\n📊 Métricas de Qualidade (Média):")
        print(f"  • Brilho:    {df_quality['brightness'].mean():.2f} ± {df_quality['brightness'].std():.2f}")
        print(f"  • Contraste: {df_quality['contrast'].mean():.2f} ± {df_quality['contrast'].std():.2f}")
        print(f"  • Nitidez:   {df_quality['sharpness'].mean():.2f} ± {df_quality['sharpness'].std():.2f}")
        
        # Gráficos de distribuição
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        axes[0].hist(df_quality['brightness'], bins=20, color='skyblue', edgecolor='black')
        axes[0].set_title('Distribuição de Brilho')
        axes[0].set_xlabel('Brilho Médio')
        
        axes[1].hist(df_quality['contrast'], bins=20, color='lightcoral', edgecolor='black')
        axes[1].set_title('Distribuição de Contraste')
        axes[1].set_xlabel('Desvio Padrão')
        
        axes[2].hist(df_quality['sharpness'], bins=20, color='lightgreen', edgecolor='black')
        axes[2].set_title('Distribuição de Nitidez')
        axes[2].set_xlabel('Variância do Laplaciano')
        
        plt.suptitle('Análise de Qualidade das Imagens Processadas', fontsize=14)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Nenhuma imagem processada para análise")

# Executar análise
if os.path.exists(OUTPUT_PATH):
    analyze_processing_quality()

In [ ]:
# Célula 16: Conclusão
print("\n" + "="*60)
print("✨ PROCESSAMENTO FINALIZADO")
print("="*60)

if os.path.exists(LOG_PATH):
    print(f"\n📁 Arquivos gerados:")
    print(f"   • Logs: {LOG_PATH}")
    
    log_files = [f for f in os.listdir(LOG_PATH) if f.endswith(('.json', '.csv'))]
    for log_file in log_files:
        file_path = os.path.join(LOG_PATH, log_file)
        file_size = os.path.getsize(file_path) / 1024  # KB
        print(f"     - {log_file} ({file_size:.1f} KB)")

if os.path.exists(OUTPUT_PATH):
    print(f"\n   • Imagens processadas: {OUTPUT_PATH}")
    
    total_processed_images = 0
    for split in ['train', 'test']:
        split_path = os.path.join(OUTPUT_PATH, split)
        if os.path.exists(split_path):
            for emotion in os.listdir(split_path):
                emotion_path = os.path.join(split_path, emotion)
                if os.path.isdir(emotion_path):
                    n_images = len([f for f in os.listdir(emotion_path) 
                                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    total_processed_images += n_images
    
    print(f"     - Total: {total_processed_images} imagens processadas")
    
    # Calcular espaço em disco
    total_size = 0
    for root, dirs, files in os.walk(OUTPUT_PATH):
        for file in files:
            file_path = os.path.join(root, file)
            total_size += os.path.getsize(file_path)
    
    total_size_mb = total_size / (1024 * 1024)
    print(f"     - Espaço utilizado: {total_size_mb:.2f} MB")

print("\n📝 Próximos passos:")
print("   1. Revisar os logs em:", LOG_PATH)
print("   2. Verificar estatísticas no CSV gerado")
print("   3. Validar qualidade das imagens processadas")
print("   4. Iniciar treinamento do modelo")

print("\n" + "="*60)
print("✅ Notebook concluído com sucesso!")
print("="*60)